In [2]:
# ---------------------------------------------------------------
# Landsat Summer Vegetation Index (NDVI or EVI), 30 m
# - For YEAR < 2013: uses Landsat 5 TM + Landsat 7 ETM+ (C02 L2 SR)
# - For YEAR >= 2013: uses Landsat 8 OLI (C02 L2 SR)
# - Computes median over Jun–Aug, scales by 10000, writes Int16 GeoTIFFs
# - New England states hardcoded
# ---------------------------------------------------------------
# Setup:
#   pip install earthengine-api
#   earthengine authenticate
# ---------------------------------------------------------------

import ee
ee.Initialize()

# ---------------- Config (edit these) ----------------
YEAR         = 2010                 # <-- set to 2010 (or any year)
INDEX        = "EVI"               # "NDVI" or "EVI"
DRIVE_FOLDER = "DATA_EE_DOWNLOAD"   # Google Drive folder
SCALE_M      = 30                   # meters
CRS          = None                 # e.g., "EPSG:5070" or None for native

NEW_ENGLAND = ["ME", "NH", "VT", "MA", "CT", "RI"]

# ------------- Helpers -------------
def mask_qapixel_strict(img):
    """Mask clouds, cloud shadow, snow, cirrus*, plus drop saturated pixels.
    (*cirrus bit is relevant for L8; zero for L5/L7)."""
    qa = img.select("QA_PIXEL")
    clear = (
        qa.bitwiseAnd(1 << 1).eq(0)  # dilated cloud
        .And(qa.bitwiseAnd(1 << 2).eq(0))  # cirrus (mostly L8)
        .And(qa.bitwiseAnd(1 << 3).eq(0))  # cloud
        .And(qa.bitwiseAnd(1 << 4).eq(0))  # cloud shadow
        .And(qa.bitwiseAnd(1 << 5).eq(0))  # snow
    )
    unsat = img.select("QA_RADSAT").eq(0)
    return img.updateMask(clear.And(unsat))

def scale_reflectance(img):
    """Apply L2 SR scale/offset to red/nir/blue, returning a copy with
    standardized band names: 'red','nir','blue' regardless of sensor."""
    scale, offset = 2.75e-05, -0.2

    # Identify sensor by available bands
    bands = img.bandNames()

    # Landsat 8/9 OLI (red=SR_B4, nir=SR_B5, blue=SR_B2)
    is_l8 = bands.contains("SR_B5")  # unique to L8/9 for NIR at B5
    red  = ee.Image(ee.Algorithms.If(is_l8,
        img.select("SR_B4").multiply(scale).add(offset),
        img.select("SR_B3").multiply(scale).add(offset)))  # L5/7 red=SR_B3
    nir  = ee.Image(ee.Algorithms.If(is_l8,
        img.select("SR_B5").multiply(scale).add(offset),
        img.select("SR_B4").multiply(scale).add(offset)))  # L5/7 nir=SR_B4
    blue = ee.Image(ee.Algorithms.If(is_l8,
        img.select("SR_B2").multiply(scale).add(offset),
        img.select("SR_B1").multiply(scale).add(offset)))  # L5/7 blue=SR_B1

    return img.addBands(
        ee.Image.cat(red.rename("red"), nir.rename("nir"), blue.rename("blue")),
        overwrite=True
    )

def add_index_float(img):
    """Compute NDVI or EVI as float in native [-1,1]ish range."""
    red = img.select("red")
    nir = img.select("nir")
    if INDEX.upper() == "EVI":
        blue = img.select("blue")
        # EVI = 2.5 * (NIR - RED) / (NIR + 6*RED - 7.5*BLUE + 1)
        num = nir.subtract(red)
        den = nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1.0)
        evi = num.divide(den).multiply(2.5).rename("vi")
        return img.addBands(evi, overwrite=True)
    else:
        # NDVI = (NIR - RED) / (NIR + RED)
        ndvi = nir.subtract(red).divide(nir.add(red)).rename("vi")
        return img.addBands(ndvi, overwrite=True)

def summer_vi_scaled(state_geom, year):
    start = ee.Date.fromYMD(year, 6, 1)
    end   = ee.Date.fromYMD(year, 9, 1)  # end exclusive

    # Choose collections by year
    if year < 2013:
        col = (ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
               .filterDate(start, end)
               .filterBounds(state_geom)
               .merge(
                   ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
                   .filterDate(start, end)
                   .filterBounds(state_geom)
               ))
    else:
        col = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
               .filterDate(start, end)
               .filterBounds(state_geom))

    # Optional scene-level cloud screen speeds things up
    col = col.filter(ee.Filter.lt("CLOUD_COVER", 70))

    # Mask → scale reflectance → compute index (float) → median → scale to Int16
    vi_median = (col
        .map(mask_qapixel_strict)
        .map(scale_reflectance)
        .map(add_index_float)
        .select("vi")
        .median()
        .clip(state_geom))

    vi_scaled = vi_median.multiply(10000).toInt16()  # [-10000,10000] approx
    # Name by index for clarity
    band_name = INDEX.lower()
    return vi_scaled.rename(band_name)

# ------------- States FC and export loop -------------
states = (ee.FeatureCollection("TIGER/2018/States")
          .filter(ee.Filter.inList("STUSPS", NEW_ENGLAND))
          .select(["NAME", "STUSPS"]))

def export_state(state_feat, year):
    name = state_feat.get("NAME").getInfo()
    code = state_feat.get("STUSPS").getInfo()
    geom = state_feat.geometry()

    vi = summer_vi_scaled(geom, year)  # Int16, scaled by 10000

    desc = f"L{ '57' if year < 2013 else '8'}_Summer_{INDEX}_{year}_{code}"
    kwargs = {
        "image": vi,
        "description": desc,
        "folder": DRIVE_FOLDER,
        "fileNamePrefix": desc,
        "region": geom,
        "scale": SCALE_M,
        "maxPixels": 1_000_000_000,
        "fileFormat": "GeoTIFF",
    }
    if CRS:
        kwargs["crs"] = CRS

    ee.batch.Export.image.toDrive(**kwargs).start()
    print(f"Started export for {name} ({code}) → {desc}")

# Kick off exports
lst = states.toList(states.size())
n   = lst.size().getInfo()
print(f"Exporting {n} New England states for YEAR={YEAR} ({INDEX}) to Drive folder '{DRIVE_FOLDER}'")
for i in range(n):
    export_state(ee.Feature(lst.get(i)), YEAR)

print("All Earth Engine export tasks started. Monitor in the Tasks tab.")


Exporting 6 New England states for YEAR=2010 (EVI) to Drive folder 'DATA_EE_DOWNLOAD'
Started export for Rhode Island (RI) → L57_Summer_EVI_2010_RI
Started export for New Hampshire (NH) → L57_Summer_EVI_2010_NH
Started export for Vermont (VT) → L57_Summer_EVI_2010_VT
Started export for Connecticut (CT) → L57_Summer_EVI_2010_CT
Started export for Maine (ME) → L57_Summer_EVI_2010_ME
Started export for Massachusetts (MA) → L57_Summer_EVI_2010_MA
All Earth Engine export tasks started. Monitor in the Tasks tab.
